# OFP Installer


## Config File Migration
We propose shifting the responsibility of maintaining config files from the 
OFP team to end users. This change aims to address the tedious and error-prone
process of users feeding back changes to the development team and waiting for
the team to provide updated config files with each release. This process 
contains a significant lag and relies on users to merge config files, which can
lead to errors.

While the OFP team will no longer be responsible for maintaining config files,
they will still play a role in migrating existing files to new versions. 
Additionally, a facility for generating a brand new set of configuration files
will be provided.

The migration process will follow a similar approach to database schema 
migrations. Config files will be processed through a sequence of migrations, 
each bringing them closer to the target version. This approach avoids the need 
for a single migration that could require a lot of unique migrations each time
a new OFP version is released. Instead, at most, we will only need a single 
new migration for each OFP version.

In most cases, the migration process should be seamless and require no input 
from users. However, there may be some cases where user input is required, 
such as when a new configuration field is added and the user needs to specify
its value.

Overall, this proposed approach should streamline the config file management
process and reduce errors, while still providing a role for the OFP team in
ensuring smooth migrations.

### Input/Output Discovery
The first step in this process is to discover the arguments needed to 
perform the install. 

1. Current Version
2. Installer Version


### Read Existing Config
The config files are in a lua format. They logically consist of a set of key
value pairs that get mapped to OFP connector values. The config
files don't directly set connectors themselves but instead are run through
a second `lua` file that will validate and `Put` values onto the connectors.

Beause the config files are lua files and are actually run through a lua 
interpreter this poses a potential problem. If we were to just read the
text of the lua file making some assumptions on what is expected as symbols
(ex. `Installed`, `NotInstalled`) we could make some mistakes. To get around
making assumptions we plan to simply use the same facilities that is used
to read the config files in the OFP and simply have our program read directly
from the resulting connector values. This will remove any doubt as to what the
config file might configure and will be exactly what was configured. This
will potentially be problematic for any `defaults` that get set when a config
key is not provided but I believe in those cases the migration will result
in a config file that explicitly sets each and every key.


In [ ]:
# psuedocode like

exclude_list

def read(from_version, to_version):
    cls = DispatchCLS()
    cls.CreateComponent('FROM', 'OFP.dll', 'NEW_CONFIG')
    cls.FROM.initialize(from_version + path_of(from_version))
    
    configuration = {k:v
                    for k,v in cls.FROM.CONFIG
                    if k not in exclude_list}

    return configuration


In [ ]:
### Migrate configuration


In [69]:
class Graph:
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.adj = {v:[] for v in vertices}
        
        for v, w in edges:
            self.add_edge(v, w)

    def add_edge(self, v, w):
        self.adj[v].append(w)

    def dfs(self, s):
        visited = set()
        stack = [s]

        while (len(stack)):
            s = stack.pop()

            if (s not in visited):
                print(s,end=' ')
                visited.add(s)

            for node in self.adj[s]:
                if (node not in visited):
                    stack.append(node)

    def shortest_path(self, node1, node2):
        path_list = [[node1]]
        path_index = 0
        
        # avoid backtracking
        visited = {node1}
        
        # edge case
        if node1 == node2:
            return path_list[0]
        
        # breadth first order 
        while path_index < len(path_list):
            current_path = path_list[path_index]
            last_node = current_path[-1]
            next_nodes = self.adj[last_node]
            
            # search goal node
            if node2 in next_nodes:
                current_path.append(node2)
                return current_path
            
            # add new paths
            for next_node in next_nodes:
                if next_node not in visited:
                    new_path = current_path[:]
                    new_path.append(next_node)
                    path_list.append(new_path)
                    visited.add(next_node)
            
            path_index += 1

        # No path is found
        return []

In [90]:
import os

files = [
    # Baseline 1
    'M1-to-M2.py',
    'M2-to-M3.py',
    'M3-to-M4.py',
    
    # Baseline 2
    'M2-to-N1.py',
    'N1-to-N2.py',
    'N2-to-N3.py',
    'N3-to-N4.py',
    'N4-to-N5.py',
    'N5-to-N6.py',
]

def get_edge_from_filename(name):
    v, w = os.path.basename(name)[:-3].split('-to-')
    return v, w

edges = [get_edge_from_filename(fname)
        for fname in files]
vertices = set(vertex
               for edge in edges
               for vertex in edge)

vertices

{'M1', 'M2', 'M3', 'M4', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6'}

In [91]:
g = Graph(vertices, edges)

g.dfs('M1')

M1 M2 N1 N2 N3 N4 N5 N6 M3 M4 

In [92]:
migration_path = g.shortest_path('M2', 'N3')
migration_path

['M2', 'N1', 'N2', 'N3']

In [94]:
migrations = [f'{_from}-to-{to}.py' 
              for _from, to in zip(migration_path[:-1], migration_path[1:])]
migrations

['M2-to-N1.py', 'N1-to-N2.py', 'N2-to-N3.py']

# Migration Graph Construction
The migration scripts will be located on disk with the following format:

```
<from versions>_<to version>.py
```

an example would be `v1_v2.py` would represent the migrations from `v1` to `v2`



## Unzip Installer

```csharp
using System.Diagnostics;
using System.IO.Compression;

//var src = @"C:\temp\python310.zip";
var src = Process.GetCurrentProcess().MainModule.FileName;
var dst = Environment.GetFolderPath(Environment.SpecialFolder.LocalApplicationData) + @"\NGC\Installer";

if (Directory.Exists(dst))
    Directory.Delete(dst, true);

Directory.CreateDirectory(dst);


using (var zip = ZipFile.OpenRead(src))
{
    foreach (ZipArchiveEntry entry in zip.Entries)
    {
        if (!entry.FullName.EndsWith(".pylauncher_exe", StringComparison.OrdinalIgnoreCase) &&
            !entry.FullName.EndsWith(@"\") &&
            !entry.FullName.EndsWith(@"/"))
        { 
            // Gets the full path to ensure that relative segments are removed.
            string destinationPath = Path.GetFullPath(Path.Combine(dst, entry.FullName));

            // Ordinal match is safest, case-sensitive volumes can be mounted within volumes that
            // are case-insensitive.
            if (destinationPath.StartsWith(dst, StringComparison.Ordinal))
            {
                Directory.GetParent(destinationPath).Create();
                entry.ExtractToFile(destinationPath);
            }
        }
    }
}

var pythoncmd = new ProcessStartInfo
{
    FileName = dst + @"\pythonw.exe",
    Arguments = $"-m startup"
};
Process.Start(pythoncmd);
```

This will take the launcher and zip file and combine them

In [ ]:
from pathlib import Path
from zipfile import ZipFile, ZIP_STORED, ZIP_DEFLATED

scriptdir = Path(__file__).resolve().parent
exepath = scriptdir / 'pylauncher.exe'
distpath = scriptdir / 'python310.zip'
outpath = scriptdir / 'installer.exe'

def create_installer():
    # Creating zipfile with exe not compressed
    # We are simply trying to add enough space for
    # the launcher. The zip container format will add some 
    # header information to the file so we will have more
    # room than we need.
    with ZipFile(outpath, mode='w', compression=ZIP_STORED) as zip:
        zip.write(exepath, arcname='.pylauncher_exe')
    
    # Now we need to merge the python embedded distribution with
    # our source
    with ZipFile(distpath, mode='r') as inzip, \
        ZipFile(outpath, mode='a', compression=ZIP_DEFLATED) as outzip:
        for name in inzip.namelist():
            outzip.writestr(name, inzip.open(name).read())
    
    # Finally we need to add our launcher on top of our archive
    with  open(exepath, mode='rb') as infile, \
        open(outpath, mode='r+b') as outfile:
        outfile.seek(0)
        outfile.write(infile.read())